# 21st_2025

In [1]:
# =============================================================================
# 21대 대통령선거 데이터 전처리 함수
# =============================================================================

import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : 깃허브 파일 URL

    Returns:
    --------
    raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xlsx"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xlsx"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_21st_presidential_election(file_path):
    """
    21대 대통령선거 데이터 전처리 함수

    엑셀 파일 구조:
    - 0~2행: 메타 정보 (무시)
    - 3행: 헤더 첫 번째 줄 (시도명, 구시군명 등)
    - 4행: 헤더 두 번째 줄 (후보자명, 득표수 등)
    - 5행부터: 실제 데이터

    Parameters:
    -----------
    file_path : 파일 경로 (로컬 파일, 깃허브 blob URL, 또는 raw URL)
        - 로컬: "./data/21대_선거결과.xlsx"
        - 깃허브 blob: "https://github.com/user/repo/blob/main/파일.xlsx" (자동 변환됨)
        - 깃허브 raw: "https://github.com/user/repo/raw/main/파일.xlsx"

    Returns:
    --------
    전처리된 선거 데이터 (pandas DataFrame, 합계 행만 포함)

    Example:
    --------
    # 깃허브에서 불러오기 (blob URL 그대로 사용 - 자동 변환됨)
    blob_url = "https://github.com/user/repo/blob/main/21대/선거결과.xlsx"
    df = process_21st_presidential_election(blob_url)

    # 로컬 파일 불러오기
    df = process_21st_presidential_election("./data/21대_선거결과.xlsx")
    """

    # ================================
    # 1. 파일 로드 (자동 감지)
    # ================================
    if file_path.startswith(('http://', 'https://')):
        # URL인 경우 - blob URL이면 raw URL로 변환
        raw_url = convert_github_url_to_raw(file_path)
        print(f"웹에서 파일 로드: {raw_url}")
        if raw_url != file_path:
            print(f"  (blob URL을 raw URL로 자동 변환)")
        df_all = pd.read_excel(raw_url, header=None)
    else:
        # 로컬 파일인 경우
        print(f"로컬에서 파일 로드: {file_path}")
        df_all = pd.read_excel(file_path, header=None)

    print(f"원본 데이터 크기: {df_all.shape}")

    # ================================
    # 2. 헤더 구성 (3행 + 4행 병합)
    # ================================
    # 파이썬 인덱스는 0부터 시작하므로:
    # 3행 = iloc[3] (실제 4번째 행)
    # 4행 = iloc[4] (실제 5번째 행)
    header_row_1 = df_all.iloc[3].fillna("")  # 시도명, 구시군명 등
    header_row_2 = df_all.iloc[4].fillna("")  # 후보자명, 득표수 등
    combined_headers = header_row_1.astype(str) + "_" + header_row_2.astype(str)

    # ================================
    # 3. 실제 데이터 추출 (5행부터)
    # ================================
    # 5행 = iloc[5] (실제 6번째 행)부터 데이터 시작
    df_data = df_all.iloc[5:].copy()
    df_data.columns = combined_headers

    print(f"데이터 추출 후 크기: {df_data.shape}")
    print(f"컬럼 수: {len(df_data.columns)}")

    # ================================
    # 4. 시도명 열 전처리
    # ================================
    # 시도명이 비어있는 행은 위 행의 값으로 채움 (forward fill)
    sido_column = [col for col in df_data.columns if "시도명" in col][0]
    df_data[sido_column] = df_data[sido_column].ffill()

    # ================================
    # 5. 주요 컬럼 추출
    # ================================
    gugun_column = [col for col in df_data.columns if "구시군명" in col][0]
    dong_column = [col for col in df_data.columns if "읍면동명" in col][0]

    print(f"주요 컬럼:")
    print(f"  - 시도: {sido_column}")
    print(f"  - 구시군: {gugun_column}")
    print(f"  - 읍면동: {dong_column}")

    # ================================
    # 6. 합계 행 필터링
    # ================================
    # '합계'가 포함된 행만 추출 (지역별 집계 데이터)
    filter_mask = (
        df_data[gugun_column].astype(str).str.contains("합계", na=False) |
        df_data[dong_column].astype(str).str.contains("합계", na=False)
    )
    df_filtered = df_data[filter_mask].copy()

    print(f"필터링 후 크기: {df_filtered.shape}")

    # ================================
    # 7. 수치형 컬럼 정리
    # ================================
    # 4번째 컬럼(인덱스 4)부터는 보통 득표수 등 수치 데이터
    numeric_columns = df_filtered.columns[4:]

    print(f"수치형 컬럼 개수: {len(numeric_columns)}")

    for col in numeric_columns:
        # 쉼표 제거 → 공백 제거 → 빈값을 0으로 → 정수형 변환
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)           # 문자열로 변환
            .str.replace(",", "")  # 천 단위 구분자 제거 (예: "1,234" → "1234")
            .str.strip()           # 앞뒤 공백 제거
            .replace("", "0")      # 빈 문자열을 "0"으로 변경
            .replace("nan", "0")   # NaN 문자열을 "0"으로 변경
            .astype(int)           # 최종적으로 정수형으로 변환
        )

    # ================================
    # 8. 결과 반환
    # ================================
    print(f"최종 데이터 크기: {df_filtered.shape}")
    print("전처리 완료!")

    return df_filtered


# =============================================================================
# 사용 예시
# =============================================================================

# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
# blob_url = "https://github.com/~~/korean-elections/blob/main/original/Presidential_Elections/21th_2025/제21대_대통령선거_개표결과.xlsx"
# df_21st = process_21st_presidential_election(blob_url)

# 로컬에서 불러오는 경우
# df_21st = process_21st_presidential_election("./data/21대_대선결과.xlsx")

# 업로드된 파일 사용하는 경우
# df_21st = process_21st_presidential_election("제21대_대통령선거_개표결과.xlsx")

# 결과 확인
# print(df_21st.head())
# print(df_21st.columns.tolist())

In [2]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url21 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Presidential_Elections/21st_2025/%EC%A0%9C21%EB%8C%80_%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0_%EA%B0%9C%ED%91%9C%EA%B2%B0%EA%B3%BC.xlsx"
df_21st = process_21st_presidential_election(blob_url21)

웹에서 파일 로드: https://github.com/sw1kwon/korean-elections/raw/main/original/Presidential_Elections/21st_2025/%EC%A0%9C21%EB%8C%80_%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0_%EA%B0%9C%ED%91%9C%EA%B2%B0%EA%B3%BC.xlsx
  (blob URL을 raw URL로 자동 변환)
원본 데이터 크기: (22692, 14)
데이터 추출 후 크기: (22687, 14)
컬럼 수: 14
주요 컬럼:
  - 시도: 시도명_
  - 구시군: 구시군명_
  - 읍면동: 읍면동명_
필터링 후 크기: (271, 14)
수치형 컬럼 개수: 10
최종 데이터 크기: (271, 14)
전처리 완료!


In [3]:
df_21st

,시도명_,구시군명_,읍면동명_,투표구명_,선거인수_,투표수_,후보자별 득표수_더불어민주당\n이재명,_국민의힘\n김문수,_개혁신당\n이준석,_민주노동당\n권영국,_무소속\n송진호,_계,무효\n투표수_,기권수_
6,전국,합계,NaN,NaN,44391871,35236497,17287513,14395639,2917523,344150,35791,34980616,255881,9155374
7,서울특별시,합계(특별시),NaN,NaN,8293885,6641606,3105459,2738405,655346,83900,5998,6589108,52498,1652279
8,서울특별시,종로구,합계,NaN,125901,99261,47735,39574,9326,1754,89,98478,783,26640
90,서울특별시,중구,합계,NaN,110181,87166,40482,36302,8450,1120,62,86416,750,23015
169,서울특별시,용산구,합계,NaN,183614,143813,58705,67927,13794,2166,102,142694,1119,39801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22256,경상남도,거창군,합계,NaN,52316,41373,12623,25259,2640,332,65,40919,454,10943
22306,경상남도,합천군,합계,NaN,37329,28935,7001,20132,1188,200,46,28567,368,8394
22365,제주특별자치도,합계(도),NaN,NaN,565255,421576,228729,145290,36909,6191,528,417647,3929,143679
22366,제주특별자치도,제주시,합계,NaN,409912,306685,169119,102224,27522,4626,384,303875,2810,103227


In [4]:
df_21st.columns.tolist()

['시도명_',
 '구시군명_',
 '읍면동명_',
 '투표구명_',
 '선거인수_',
 '투표수_',
 '후보자별 득표수_더불어민주당\n이재명',
 '_국민의힘\n김문수',
 '_개혁신당\n이준석',
 '_민주노동당\n권영국',
 '_무소속\n송진호',
 '_계',
 '무효\n투표수_',
 '기권수_']

In [5]:
rename_dict21 = {
    '시도명_': '시도',
    '구시군명_': '구시군',
    '선거인수_': '선거인수',
    '투표수_': '투표수',
    '후보자별 득표수_더불어민주당\n이재명': '득표수_1_더불어민주당_이재명',
    '_국민의힘\n김문수': '득표수_2_국민의힘_김문수',
    '_개혁신당\n이준석': '득표수_4_개혁신당_이준석',
    '_민주노동당\n권영국': '득표수_5_민주노동당_권영국',
    '_무소속\n송진호': '득표수_8_무소속_송진호',
    '_계': '득표수_계',
    '무효\n투표수_': '무효투표수',
    '기권수_': '기권수'
}

In [6]:
df_21st.rename(columns=rename_dict21).drop(columns=['읍면동명_','투표구명_'])

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_김문수,득표수_4_개혁신당_이준석,득표수_5_민주노동당_권영국,득표수_8_무소속_송진호,득표수_계,무효투표수,기권수
6,전국,합계,44391871,35236497,17287513,14395639,2917523,344150,35791,34980616,255881,9155374
7,서울특별시,합계(특별시),8293885,6641606,3105459,2738405,655346,83900,5998,6589108,52498,1652279
8,서울특별시,종로구,125901,99261,47735,39574,9326,1754,89,98478,783,26640
90,서울특별시,중구,110181,87166,40482,36302,8450,1120,62,86416,750,23015
169,서울특별시,용산구,183614,143813,58705,67927,13794,2166,102,142694,1119,39801
...,...,...,...,...,...,...,...,...,...,...,...,...
22256,경상남도,거창군,52316,41373,12623,25259,2640,332,65,40919,454,10943
22306,경상남도,합천군,37329,28935,7001,20132,1188,200,46,28567,368,8394
22365,제주특별자치도,합계(도),565255,421576,228729,145290,36909,6191,528,417647,3929,143679
22366,제주특별자치도,제주시,409912,306685,169119,102224,27522,4626,384,303875,2810,103227


In [7]:
df_21st = df_21st.rename(columns=rename_dict21).drop(columns=['읍면동명_','투표구명_'])

In [8]:
# 구시군 원소 괄호 제거
df_21st['구시군'] = (
    df_21st['구시군']
    .str.replace(r"\(.*\)", "", regex=True)
    .str.strip()
)

In [9]:
df_21st

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_김문수,득표수_4_개혁신당_이준석,득표수_5_민주노동당_권영국,득표수_8_무소속_송진호,득표수_계,무효투표수,기권수
6,전국,합계,44391871,35236497,17287513,14395639,2917523,344150,35791,34980616,255881,9155374
7,서울특별시,합계,8293885,6641606,3105459,2738405,655346,83900,5998,6589108,52498,1652279
8,서울특별시,종로구,125901,99261,47735,39574,9326,1754,89,98478,783,26640
90,서울특별시,중구,110181,87166,40482,36302,8450,1120,62,86416,750,23015
169,서울특별시,용산구,183614,143813,58705,67927,13794,2166,102,142694,1119,39801
...,...,...,...,...,...,...,...,...,...,...,...,...
22256,경상남도,거창군,52316,41373,12623,25259,2640,332,65,40919,454,10943
22306,경상남도,합천군,37329,28935,7001,20132,1188,200,46,28567,368,8394
22365,제주특별자치도,합계,565255,421576,228729,145290,36909,6191,528,417647,3929,143679
22366,제주특별자치도,제주시,409912,306685,169119,102224,27522,4626,384,303875,2810,103227


In [10]:
df_21st[df_21st['구시군'] == '합계']

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_김문수,득표수_4_개혁신당_이준석,득표수_5_민주노동당_권영국,득표수_8_무소속_송진호,득표수_계,무효투표수,기권수
6,전국,합계,44391871,35236497,17287513,14395639,2917523,344150,35791,34980616,255881,9155374
7,서울특별시,합계,8293885,6641606,3105459,2738405,655346,83900,5998,6589108,52498,1652279
3245,부산광역시,합계,2865552,2245755,895213,1146238,168473,18189,2099,2230212,15543,619797
4652,대구광역시,합계,2049078,1643051,379130,1103913,135376,12531,1362,1632312,10739,406027
5659,인천광역시,합계,2619348,2035355,1044295,776952,176739,20743,2098,2020827,14528,583993
6764,광주광역시,합계,1194471,1002149,844682,79937,62104,8767,934,996424,5725,192322
7339,대전광역시,합계,1241882,977609,470321,393549,94724,9905,1109,969608,8001,264273
7892,울산광역시,합계,934509,747950,315820,353180,63177,9299,899,742375,5575,186559
8297,세종특별자치시,합계,307067,254695,140620,83965,25004,2961,235,252785,1910,52372
8437,경기도,합계,11715343,9297448,4821148,3504620,816435,84074,8356,9234633,62815,2417895


In [11]:
df_21st.to_csv("temp1_president_21.csv", index=False, encoding="utf-8-sig")

# 20th_2022

In [12]:
# =============================================================================
# 20대 대통령선거 데이터 전처리 함수
# =============================================================================

import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : 깃허브 파일 URL

    Returns:
    --------
    raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xlsx"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xlsx"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_20th_presidential_election(file_path):
    """
    20대 대통령선거 데이터 전처리 함수

    엑셀 파일 구조:
    - 0행: 헤더 (시도명, 구시군, 읍면동명, 후보자명 등)
    - 1행부터: 실제 데이터

    Parameters:
    -----------
    file_path : 파일 경로 (로컬 파일, 깃허브 blob URL, 또는 raw URL)
        - 로컬: "./data/20대_선거결과.xlsx"
        - 깃허브 blob: "https://github.com/user/repo/blob/main/파일.xlsx" (자동 변환됨)
        - 깃허브 raw: "https://github.com/user/repo/raw/main/파일.xlsx"

    Returns:
    --------
    전처리된 선거 데이터 (pandas DataFrame, 합계 행만 포함)

    Example:
    --------
    # 깃허브에서 불러오기 (blob URL 그대로 사용 - 자동 변환됨)
    blob_url = "https://github.com/user/repo/blob/main/20대/선거결과.xlsx"
    df = process_20th_presidential_election(blob_url)

    # 로컬 파일 불러오기
    df = process_20th_presidential_election("./data/20대_선거결과.xlsx")
    """

    # ================================
    # 1. 파일 로드 (자동 감지)
    # ================================
    if file_path.startswith(('http://', 'https://')):
        # URL인 경우 - blob URL이면 raw URL로 변환
        raw_url = convert_github_url_to_raw(file_path)
        print(f"웹에서 파일 로드: {raw_url}")
        if raw_url != file_path:
            print(f"  (blob URL을 raw URL로 자동 변환)")
        df_all = pd.read_excel(raw_url, header=0)  # 첫 번째 행이 헤더
    else:
        # 로컬 파일인 경우
        print(f"로컬에서 파일 로드: {file_path}")
        df_all = pd.read_excel(file_path, header=0)  # 첫 번째 행이 헤더

    print(f"원본 데이터 크기: {df_all.shape}")
    print(f"컬럼명: {list(df_all.columns)}")

    # ================================
    # 2. 주요 컬럼 확인
    # ================================
    # 20대는 헤더가 단순함 (0행이 바로 컬럼명)
    gugun_column = '구시군'
    dong_column = '읍면동명'

    print(f"주요 컬럼:")
    print(f"  - 구시군: {gugun_column}")
    print(f"  - 읍면동: {dong_column}")

    # ================================
    # 3. 합계 행 필터링
    # ================================
    # '합계'가 포함된 행만 추출 (지역별 집계 데이터)
    filter_mask = (
        df_all[gugun_column].astype(str).str.contains("합계", na=False) |
        df_all[dong_column].astype(str).str.contains("합계", na=False)
    )
    df_filtered = df_all[filter_mask].copy()

    print(f"필터링 후 크기: {df_filtered.shape}")

    # ================================
    # 4. 수치형 컬럼 정리
    # ================================
    # 4번째 컬럼(인덱스 4)부터는 보통 득표수 등 수치 데이터
    numeric_columns = df_filtered.columns[4:]

    print(f"수치형 컬럼 개수: {len(numeric_columns)}")

    for col in numeric_columns:
        # 쉼표 제거 → 공백 제거 → 빈값을 0으로 → 정수형 변환
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)           # 문자열로 변환
            .str.replace(",", "")  # 천 단위 구분자 제거 (예: "1,234" → "1234")
            .str.strip()           # 앞뒤 공백 제거
            .replace("", "0")      # 빈 문자열을 "0"으로 변경
            .replace("nan", "0")   # NaN 문자열을 "0"으로 변경
            .astype(int)           # 최종적으로 정수형으로 변환
        )

    # ================================
    # 5. 결과 반환
    # ================================
    print(f"최종 데이터 크기: {df_filtered.shape}")
    print("전처리 완료!")

    return df_filtered


# =============================================================================
# 사용 예시
# =============================================================================

# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
# blob_url = "https://github.com/~~/korean-elections/blob/main/original/Presidential_Elections/20th_2022/개표단위별_개표결과_대통령선거_전체.xlsx"
# df_20th = process_20th_presidential_election(blob_url)

# 로컬에서 불러오는 경우
# df_20th = process_20th_presidential_election("./data/20대_대선결과.xlsx")

# 업로드된 파일 사용하는 경우
# df_20th = process_20th_presidential_election("개표단위별_개표결과_대통령선거_전체.xlsx")

# 결과 확인
# print(df_20th.head())
# print(df_20th.columns.tolist())

In [13]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url20 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Presidential_Elections/20th_2022/%EA%B0%9C%ED%91%9C%EB%8B%A8%EC%9C%84%EB%B3%84_%EA%B0%9C%ED%91%9C%EA%B2%B0%EA%B3%BC_%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0_%EC%A0%84%EC%B2%B4.xlsx"
df_20th = process_20th_presidential_election(blob_url20)

웹에서 파일 로드: https://github.com/sw1kwon/korean-elections/raw/main/original/Presidential_Elections/20th_2022/%EA%B0%9C%ED%91%9C%EB%8B%A8%EC%9C%84%EB%B3%84_%EA%B0%9C%ED%91%9C%EA%B2%B0%EA%B3%BC_%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0_%EC%A0%84%EC%B2%B4.xlsx
  (blob URL을 raw URL로 자동 변환)
원본 데이터 크기: (22753, 21)
컬럼명: ['시도', '구시군', '읍면동명', '투표구명', '선거인수', '투표수', '더불어민주당\n이재명', '국민의힘\n윤석열', '정의당\n심상정', '기본소득당\n오준호', '국가혁명당 \n허경영', '노동당\n이백윤', '새누리당\n옥은호', '신자유민주연합\n김경재', '우리공화당\n조원진', '진보당\n김재연', '통일한국당\n이경희', '한류연합당\n김민찬', '계', '무효투표수', '기권수']
주요 컬럼:
  - 구시군: 구시군
  - 읍면동: 읍면동명
필터링 후 크기: (268, 21)
수치형 컬럼 개수: 17
최종 데이터 크기: (268, 21)
전처리 완료!


In [14]:
df_20th

,시도,구시군,읍면동명,투표구명,선거인수,투표수,더불어민주당\n이재명,국민의힘\n윤석열,정의당\n심상정,기본소득당\n오준호,...,노동당\n이백윤,새누리당\n옥은호,신자유민주연합\n김경재,우리공화당\n조원진,진보당\n김재연,통일한국당\n이경희,한류연합당\n김민찬,계,무효투표수,기권수
0,전국,합계,NaN,NaN,44197692,34067853,16147738,16394815,803358,18105,...,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계(특별시),NaN,NaN,8346647,6501831,2944981,3255747,180324,3829,...,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
2,서울특별시,종로구,합계,NaN,129968,100629,46130,49172,3115,53,...,42,9,50,82,105,20,30,99366,1263,29339
85,서울특별시,중구,합계,NaN,111448,84998,38244,42906,2310,46,...,19,10,17,67,57,17,28,84184,814,26450
164,서울특별시,용산구,합계,NaN,199077,152068,60063,85047,4374,67,...,37,24,56,86,100,27,46,150682,1386,47009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,경상남도,거창군,합계,NaN,53049,41399,11963,27254,895,46,...,19,18,12,42,92,62,52,40945,454,11650
22365,경상남도,합천군,합계,NaN,39768,31270,6911,22742,511,30,...,24,9,16,62,96,68,52,30832,438,8498
22426,제주특별자치도,합계(도),NaN,NaN,564354,409649,213130,173014,13598,267,...,146,63,101,271,571,228,310,405238,4411,154705
22427,제주특별자치도,제주시,합계,NaN,408552,296826,157695,122084,10007,189,...,108,46,80,193,395,159,197,293649,3177,111726


In [15]:
df_20th.columns.tolist()

['시도',
 '구시군',
 '읍면동명',
 '투표구명',
 '선거인수',
 '투표수',
 '더불어민주당\n이재명',
 '국민의힘\n윤석열',
 '정의당\n심상정',
 '기본소득당\n오준호',
 '국가혁명당 \n허경영',
 '노동당\n이백윤',
 '새누리당\n옥은호',
 '신자유민주연합\n김경재',
 '우리공화당\n조원진',
 '진보당\n김재연',
 '통일한국당\n이경희',
 '한류연합당\n김민찬',
 '계',
 '무효투표수',
 '기권수']

In [16]:
rename_dict20 = {
    '더불어민주당\n이재명': '득표수_1_더불어민주당_이재명',
    '국민의힘\n윤석열': '득표수_2_국민의힘_윤석열',
    '정의당\n심상정': '득표수_3_정의당_심상정',
    '기본소득당\n오준호': '득표수_5_기본소득당_오준호',
    '국가혁명당 \n허경영': '득표수_6_국가혁명당_허경영',
    '노동당\n이백윤': '득표수_7_노동당_이백윤',
    '새누리당\n옥은호': '득표수_8_새누리당_옥은호',
    '신자유민주연합\n김경재': '득표수_10_신자유민주연합_김경재',
    '우리공화당\n조원진': '득표수_11_우리공화당_조원진',
    '진보당\n김재연': '득표수_12_진보당_김재연',
    '통일한국당\n이경희': '득표수_13_통일한국당_이경희',
    '한류연합당\n김민찬': '득표수_14_한류연합당_김민찬',
    '계': '득표수_계'
}

In [17]:
df_20th.rename(columns=rename_dict20).drop(columns=['읍면동명','투표구명'])

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_윤석열,득표수_3_정의당_심상정,득표수_5_기본소득당_오준호,득표수_6_국가혁명당_허경영,득표수_7_노동당_이백윤,득표수_8_새누리당_옥은호,득표수_10_신자유민주연합_김경재,득표수_11_우리공화당_조원진,득표수_12_진보당_김재연,득표수_13_통일한국당_이경희,득표수_14_한류연합당_김민찬,득표수_계,무효투표수,기권수
0,전국,합계,44197692,34067853,16147738,16394815,803358,18105,281481,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계(특별시),8346647,6501831,2944981,3255747,180324,3829,36540,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
2,서울특별시,종로구,129968,100629,46130,49172,3115,53,558,42,9,50,82,105,20,30,99366,1263,29339
85,서울특별시,중구,111448,84998,38244,42906,2310,46,463,19,10,17,67,57,17,28,84184,814,26450
164,서울특별시,용산구,199077,152068,60063,85047,4374,67,755,37,24,56,86,100,27,46,150682,1386,47009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,경상남도,거창군,53049,41399,11963,27254,895,46,490,19,18,12,42,92,62,52,40945,454,11650
22365,경상남도,합천군,39768,31270,6911,22742,511,30,311,24,9,16,62,96,68,52,30832,438,8498
22426,제주특별자치도,합계(도),564354,409649,213130,173014,13598,267,3539,146,63,101,271,571,228,310,405238,4411,154705
22427,제주특별자치도,제주시,408552,296826,157695,122084,10007,189,2496,108,46,80,193,395,159,197,293649,3177,111726


In [18]:
df_20th = df_20th.rename(columns=rename_dict20).drop(columns=['읍면동명','투표구명'])

In [19]:
# 구시군 원소 괄호 제거
df_20th['구시군'] = (
    df_20th['구시군']
    .str.replace(r"\(.*\)", "", regex=True)
    .str.strip()
)

In [20]:
df_20th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_윤석열,득표수_3_정의당_심상정,득표수_5_기본소득당_오준호,득표수_6_국가혁명당_허경영,득표수_7_노동당_이백윤,득표수_8_새누리당_옥은호,득표수_10_신자유민주연합_김경재,득표수_11_우리공화당_조원진,득표수_12_진보당_김재연,득표수_13_통일한국당_이경희,득표수_14_한류연합당_김민찬,득표수_계,무효투표수,기권수
0,전국,합계,44197692,34067853,16147738,16394815,803358,18105,281481,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계,8346647,6501831,2944981,3255747,180324,3829,36540,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
2,서울특별시,종로구,129968,100629,46130,49172,3115,53,558,42,9,50,82,105,20,30,99366,1263,29339
85,서울특별시,중구,111448,84998,38244,42906,2310,46,463,19,10,17,67,57,17,28,84184,814,26450
164,서울특별시,용산구,199077,152068,60063,85047,4374,67,755,37,24,56,86,100,27,46,150682,1386,47009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,경상남도,거창군,53049,41399,11963,27254,895,46,490,19,18,12,42,92,62,52,40945,454,11650
22365,경상남도,합천군,39768,31270,6911,22742,511,30,311,24,9,16,62,96,68,52,30832,438,8498
22426,제주특별자치도,합계,564354,409649,213130,173014,13598,267,3539,146,63,101,271,571,228,310,405238,4411,154705
22427,제주특별자치도,제주시,408552,296826,157695,122084,10007,189,2496,108,46,80,193,395,159,197,293649,3177,111726


In [21]:
df_20th[df_20th['구시군'] == '합계']

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이재명,득표수_2_국민의힘_윤석열,득표수_3_정의당_심상정,득표수_5_기본소득당_오준호,득표수_6_국가혁명당_허경영,득표수_7_노동당_이백윤,득표수_8_새누리당_옥은호,득표수_10_신자유민주연합_김경재,득표수_11_우리공화당_조원진,득표수_12_진보당_김재연,득표수_13_통일한국당_이경희,득표수_14_한류연합당_김민찬,득표수_계,무효투표수,기권수
0,전국,합계,44197692,34067853,16147738,16394815,803358,18105,281481,9176,4970,8317,25972,37366,11708,17305,33760311,307542,10129839
1,서울특별시,합계,8346647,6501831,2944981,3255747,180324,3829,36540,1571,844,1791,4657,5615,1333,1907,6439139,62692,1844816
3242,부산광역시,합계,2921510,2200224,831896,1270072,47541,1071,21990,546,352,527,1867,2799,575,942,2180178,20046,721286
4651,대구광역시,합계,2046714,1611512,345045,1199888,31131,892,13941,344,261,451,2824,938,472,619,1596806,14706,435202
5612,인천광역시,합계,2519225,1883504,913320,878560,51852,1116,16733,508,276,449,1378,1593,511,758,1867054,16450,635721
6703,광주광역시,합계,1209206,985492,830058,124511,14865,434,6138,242,92,140,112,1366,188,455,978601,6891,223714
7290,대전광역시,합계,1233177,945308,434950,464060,25445,566,8593,223,138,227,588,958,258,395,936401,8907,287869
7844,울산광역시,합계,942210,735461,297134,396321,21292,375,9234,308,109,185,685,2180,234,333,728390,7071,206749
8263,세종특별자치시,합계,288895,231832,119349,101491,6780,100,1594,50,23,48,121,181,66,88,229891,1941,57063
8396,경기도,합계,11433288,8763727,4428151,3965341,205709,4151,63207,1919,1124,1990,5897,8768,1927,3192,8691376,72351,2669561


In [22]:
df_20th.to_csv("temp1_president_20.csv", index=False, encoding="utf-8-sig")

# 19th_2017

In [23]:
# =============================================================================
# 19대 대통령선거 데이터 전처리 함수
# =============================================================================

import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : 깃허브 파일 URL

    Returns:
    --------
    raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xlsx"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xlsx"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_19th_presidential_election(file_path):
    """
    19대 대통령선거 데이터 전처리 함수

    엑셀 파일 구조:
    - 0행: 헤더 첫 번째 줄 (시도명, 구시군명 등)
    - 1행: 헤더 두 번째 줄 (후보자명, 득표수 등)
    - 2행부터: 실제 데이터

    Parameters:
    -----------
    file_path : 파일 경로 (로컬 파일, 깃허브 blob URL, 또는 raw URL)
        - 로컬: "./data/19대_선거결과.xlsx"
        - 깃허브 blob: "https://github.com/user/repo/blob/main/파일.xlsx" (자동 변환됨)
        - 깃허브 raw: "https://github.com/user/repo/raw/main/파일.xlsx"

    Returns:
    --------
    전처리된 선거 데이터 (pandas DataFrame, 합계 행만 포함)

    Example:
    --------
    # 깃허브에서 불러오기 (blob URL 그대로 사용 - 자동 변환됨)
    blob_url = "https://github.com/user/repo/blob/main/19대/선거결과.xlsx"
    df = process_19th_presidential_election(blob_url)

    # 로컬 파일 불러오기
    df = process_19th_presidential_election("./data/19대_선거결과.xlsx")
    """

    # ================================
    # 1. 파일 로드 (자동 감지)
    # ================================
    if file_path.startswith(('http://', 'https://')):
        # URL인 경우 - blob URL이면 raw URL로 변환
        raw_url = convert_github_url_to_raw(file_path)
        print(f"웹에서 파일 로드: {raw_url}")
        if raw_url != file_path:
            print(f"  (blob URL을 raw URL로 자동 변환)")
        df_all = pd.read_excel(raw_url, header=None)
    else:
        # 로컬 파일인 경우
        print(f"로컬에서 파일 로드: {file_path}")
        df_all = pd.read_excel(file_path, header=None)

    print(f"원본 데이터 크기: {df_all.shape}")

    # ================================
    # 2. 헤더 구성 (0행 + 1행 병합)
    # ================================
    # 파이썬 인덱스는 0부터 시작하므로:
    # 0행 = iloc[0] (실제 1번째 행)
    # 1행 = iloc[1] (실제 2번째 행)
    header_row_1 = df_all.iloc[0].fillna("")  # 시도명, 구시군명 등
    header_row_2 = df_all.iloc[1].fillna("")  # 후보자명, 득표수 등
    combined_headers = header_row_1.astype(str) + "_" + header_row_2.astype(str)

    # ================================
    # 3. 실제 데이터 추출 (2행부터)
    # ================================
    # 2행 = iloc[2] (실제 3번째 행)부터 데이터 시작
    df_data = df_all.iloc[2:].copy()
    df_data.columns = combined_headers

    print(f"데이터 추출 후 크기: {df_data.shape}")
    print(f"컬럼 수: {len(df_data.columns)}")

    # ================================
    # 4. 주요 컬럼 추출
    # ================================
    sido_column = [col for col in df_data.columns if "시도명" in col][0]
    gugun_column = [col for col in df_data.columns if "구시군명" in col][0]
    dong_column = [col for col in df_data.columns if "읍면동명" in col][0]

    print(f"주요 컬럼:")
    print(f"  - 시도: {sido_column}")
    print(f"  - 구시군: {gugun_column}")
    print(f"  - 읍면동: {dong_column}")

    # ================================
    # 5. 합계 행 필터링
    # ================================
    # '전국', '합계'가 포함된 행만 추출 (지역별 집계 데이터)
    # 19대는 시도명에 '전국'도 포함하여 필터링
    filter_mask = (
        df_data[sido_column].astype(str).str.contains("전국", na=False) |
        df_data[gugun_column].astype(str).str.contains("합계", na=False) |
        df_data[dong_column].astype(str).str.contains("합계", na=False)
    )
    df_filtered = df_data[filter_mask].copy()

    print(f"필터링 후 크기: {df_filtered.shape}")

    # ================================
    # 6. 수치형 컬럼 정리
    # ================================
    # 4번째 컬럼(인덱스 4)부터는 보통 득표수 등 수치 데이터
    numeric_columns = df_filtered.columns[4:]

    print(f"수치형 컬럼 개수: {len(numeric_columns)}")

    for col in numeric_columns:
        # 쉼표 제거 → 공백 제거 → 빈값을 0으로 → 정수형 변환
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)           # 문자열로 변환
            .str.replace(",", "")  # 천 단위 구분자 제거 (예: "1,234" → "1234")
            .str.strip()           # 앞뒤 공백 제거
            .replace("", "0")      # 빈 문자열을 "0"으로 변경
            .replace("nan", "0")   # NaN 문자열을 "0"으로 변경
            .astype(int)           # 최종적으로 정수형으로 변환
        )

    # ================================
    # 7. 결과 반환
    # ================================
    print(f"최종 데이터 크기: {df_filtered.shape}")
    print("전처리 완료!")

    return df_filtered


# =============================================================================
# 사용 예시
# =============================================================================

# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
# blob_url = "https://github.com/~~/korean-elections/blob/main/original/Presidential_Elections/19th_2017/제19대 대통령선거 개표자료.xlsx"
# df_19th = process_19th_presidential_election(blob_url)

# 로컬에서 불러오는 경우
# df_19th = process_19th_presidential_election("./data/19대_대선결과.xlsx")

# 업로드된 파일 사용하는 경우
# df_19th = process_19th_presidential_election("제19대 대통령선거 개표자료.xlsx")

# 결과 확인
# print(df_19th.head())
# print(df_19th.columns.tolist())

In [24]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url19 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Presidential_Elections/19th_2017/%EC%A0%9C19%EB%8C%80%20%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0%20%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C.xlsx"
df_19th = process_19th_presidential_election(blob_url19)

웹에서 파일 로드: https://github.com/sw1kwon/korean-elections/raw/main/original/Presidential_Elections/19th_2017/%EC%A0%9C19%EB%8C%80%20%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0%20%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C.xlsx
  (blob URL을 raw URL로 자동 변환)
원본 데이터 크기: (22216, 22)
데이터 추출 후 크기: (22214, 22)
컬럼 수: 22
주요 컬럼:
  - 시도: 시도명_
  - 구시군: 구시군명_
  - 읍면동: 읍면동명_
필터링 후 크기: (268, 22)
수치형 컬럼 개수: 18
최종 데이터 크기: (268, 22)
전처리 완료!


In [25]:
df_19th

,시도명_,구시군명_,읍면동명_,투표구명_,선거인수_,투표수_,후보자별 득표수_더불어민주당\n문재인,_자유한국당\n홍준표,_국민의당\n안철수,_바른정당\n유승민,...,_경제애국당\n오영국,_국민대통합당\n장성민,_늘푸른한국당\n이재오,_민중연합당\n김선동,_한국국민당\n이경희,_홍익당\n윤홍식,_무소속\n김민찬,_계,무효투표수_,기권수_
2,전국,NaN,NaN,NaN,42479710,32807908,13423800,7852849,6998342,2208771,...,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802
3,서울특별시,합계,NaN,NaN,8382999,6590646,2781345,1365285,1492767,476973,...,789,3554,1938,3416,1277,2177,3950,6568917,21729,1792353
4,서울특별시,종로구,합계,NaN,133769,102566,42512,22325,22313,7412,...,5,78,31,63,26,47,49,102202,364,31203
86,서울특별시,중구,합계,NaN,109836,82852,34062,17901,19372,5874,...,12,53,21,55,18,25,51,82595,257,26984
165,서울특별시,용산구,합계,NaN,197962,148157,58081,35230,32109,11825,...,17,68,23,72,32,51,80,147660,497,49805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775,경상남도,거창군,합계,NaN,53345,41325,11256,19976,4923,2447,...,12,37,22,48,54,105,100,40949,376,12020
21825,경상남도,합천군,합계,NaN,42887,33021,7143,19699,3077,1317,...,23,26,16,28,51,153,76,32711,310,9866
21889,제주특별자치도,합계,NaN,NaN,518000,374459,169493,68063,77861,22784,...,269,312,113,335,236,289,556,372412,2047,143541
21890,제주특별자치도,제주시,합계,NaN,375292,273163,125717,48027,55971,16885,...,96,193,73,257,153,178,368,271780,1383,102129


In [26]:
df_19th.columns.tolist()

['시도명_',
 '구시군명_',
 '읍면동명_',
 '투표구명_',
 '선거인수_',
 '투표수_',
 '후보자별 득표수_더불어민주당\n문재인',
 '_자유한국당\n홍준표',
 '_국민의당\n안철수',
 '_바른정당\n유승민',
 '_정의당\n심상정',
 '_새누리당\n조원진',
 '_경제애국당\n오영국',
 '_국민대통합당\n장성민',
 '_늘푸른한국당\n이재오',
 '_민중연합당\n김선동',
 '_한국국민당\n이경희',
 '_홍익당\n윤홍식',
 '_무소속\n김민찬',
 '_계',
 '무효투표수_',
 '기권수_']

In [27]:
rename_dict19 = {
    '시도명_': '시도',
    '구시군명_': '구시군',
    '선거인수_': '선거인수',
    '투표수_': '투표수',
    '후보자별 득표수_더불어민주당\n문재인': '득표수_1_더불어민주당_문재인',
    '_자유한국당\n홍준표': '득표수_2_자유한국당_홍준표',
    '_국민의당\n안철수': '득표수_3_국민의당_안철수',
    '_바른정당\n유승민': '득표수_4_바른정당_유승민',
    '_정의당\n심상정': '득표수_5_정의당_심상정',
    '_새누리당\n조원진': '득표수_6_새누리당_조원진',
    '_경제애국당\n오영국': '득표수_7_경제애국당_오영국',
    '_국민대통합당\n장성민': '득표수_8_국민대통합당_장성민',
    '_늘푸른한국당\n이재오': '득표수_9_늘푸른한국당_이재오',
    '_민중연합당\n김선동': '득표수_10_민중연합당_김선동',
    '_한국국민당\n이경희': '득표수_12_한국국민당_이경희',
    '_홍익당\n윤홍식': '득표수_14_홍익당_윤홍식',
    '_무소속\n김민찬': '득표수_15_무소속_김민찬',
    '_계': '득표수_계',
    '무효투표수_': '무효투표수',
    '기권수_': '기권수'
}

In [28]:
df_19th.rename(columns=rename_dict19).drop(columns=['읍면동명_','투표구명_'])

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문재인,득표수_2_자유한국당_홍준표,득표수_3_국민의당_안철수,득표수_4_바른정당_유승민,득표수_5_정의당_심상정,득표수_6_새누리당_조원진,득표수_7_경제애국당_오영국,득표수_8_국민대통합당_장성민,득표수_9_늘푸른한국당_이재오,득표수_10_민중연합당_김선동,득표수_12_한국국민당_이경희,득표수_14_홍익당_윤홍식,득표수_15_무소속_김민찬,득표수_계,무효투표수,기권수
2,전국,NaN,42479710,32807908,13423800,7852849,6998342,2208771,2017458,42949,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802
3,서울특별시,합계,8382999,6590646,2781345,1365285,1492767,476973,425459,9987,789,3554,1938,3416,1277,2177,3950,6568917,21729,1792353
4,서울특별시,종로구,133769,102566,42512,22325,22313,7412,7113,228,5,78,31,63,26,47,49,102202,364,31203
86,서울특별시,중구,109836,82852,34062,17901,19372,5874,4993,158,12,53,21,55,18,25,51,82595,257,26984
165,서울특별시,용산구,197962,148157,58081,35230,32109,11825,9773,299,17,68,23,72,32,51,80,147660,497,49805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775,경상남도,거창군,53345,41325,11256,19976,4923,2447,1923,46,12,37,22,48,54,105,100,40949,376,12020
21825,경상남도,합천군,42887,33021,7143,19699,3077,1317,1065,37,23,26,16,28,51,153,76,32711,310,9866
21889,제주특별자치도,합계,518000,374459,169493,68063,77861,22784,31716,385,269,312,113,335,236,289,556,372412,2047,143541
21890,제주특별자치도,제주시,375292,273163,125717,48027,55971,16885,23592,270,96,193,73,257,153,178,368,271780,1383,102129


In [29]:
df_19th = df_19th.rename(columns=rename_dict19).drop(columns=['읍면동명_','투표구명_'])

In [30]:
df_19th[df_19th['구시군'].isna()]

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문재인,득표수_2_자유한국당_홍준표,득표수_3_국민의당_안철수,득표수_4_바른정당_유승민,득표수_5_정의당_심상정,득표수_6_새누리당_조원진,득표수_7_경제애국당_오영국,득표수_8_국민대통합당_장성민,득표수_9_늘푸른한국당_이재오,득표수_10_민중연합당_김선동,득표수_12_한국국민당_이경희,득표수_14_홍익당_윤홍식,득표수_15_무소속_김민찬,득표수_계,무효투표수,기권수
2,전국,NaN,42479710,32807908,13423800,7852849,6998342,2208771,2017458,42949,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802


In [31]:
df_19th['구시군'] = df_19th['구시군'].fillna('합계')

In [32]:
df_19th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문재인,득표수_2_자유한국당_홍준표,득표수_3_국민의당_안철수,득표수_4_바른정당_유승민,득표수_5_정의당_심상정,득표수_6_새누리당_조원진,득표수_7_경제애국당_오영국,득표수_8_국민대통합당_장성민,득표수_9_늘푸른한국당_이재오,득표수_10_민중연합당_김선동,득표수_12_한국국민당_이경희,득표수_14_홍익당_윤홍식,득표수_15_무소속_김민찬,득표수_계,무효투표수,기권수
2,전국,합계,42479710,32807908,13423800,7852849,6998342,2208771,2017458,42949,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802
3,서울특별시,합계,8382999,6590646,2781345,1365285,1492767,476973,425459,9987,789,3554,1938,3416,1277,2177,3950,6568917,21729,1792353
4,서울특별시,종로구,133769,102566,42512,22325,22313,7412,7113,228,5,78,31,63,26,47,49,102202,364,31203
86,서울특별시,중구,109836,82852,34062,17901,19372,5874,4993,158,12,53,21,55,18,25,51,82595,257,26984
165,서울특별시,용산구,197962,148157,58081,35230,32109,11825,9773,299,17,68,23,72,32,51,80,147660,497,49805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775,경상남도,거창군,53345,41325,11256,19976,4923,2447,1923,46,12,37,22,48,54,105,100,40949,376,12020
21825,경상남도,합천군,42887,33021,7143,19699,3077,1317,1065,37,23,26,16,28,51,153,76,32711,310,9866
21889,제주특별자치도,합계,518000,374459,169493,68063,77861,22784,31716,385,269,312,113,335,236,289,556,372412,2047,143541
21890,제주특별자치도,제주시,375292,273163,125717,48027,55971,16885,23592,270,96,193,73,257,153,178,368,271780,1383,102129


In [33]:
df_19th[df_19th['구시군'] == '합계']

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_문재인,득표수_2_자유한국당_홍준표,득표수_3_국민의당_안철수,득표수_4_바른정당_유승민,득표수_5_정의당_심상정,득표수_6_새누리당_조원진,득표수_7_경제애국당_오영국,득표수_8_국민대통합당_장성민,득표수_9_늘푸른한국당_이재오,득표수_10_민중연합당_김선동,득표수_12_한국국민당_이경희,득표수_14_홍익당_윤홍식,득표수_15_무소속_김민찬,득표수_계,무효투표수,기권수
2,전국,합계,42479710,32807908,13423800,7852849,6998342,2208771,2017458,42949,6040,21709,9140,27229,11355,18543,33990,32672175,135733,9671802
3,서울특별시,합계,8382999,6590646,2781345,1365285,1492767,476973,425459,9987,789,3554,1938,3416,1277,2177,3950,6568917,21729,1792353
3226,부산광역시,합계,2950224,2261633,872127,720484,378907,162480,109329,2651,276,1316,465,981,496,1041,2156,2252709,8924,688591
4623,대구광역시,합계,2043276,1581347,342620,714205,235757,198459,74440,4057,259,563,324,804,401,986,1501,1574376,6971,461929
5559,인천광역시,합계,2409031,1820091,747090,379191,428888,118691,129925,2646,374,1618,410,1230,594,625,1681,1812963,7128,588940
6600,광주광역시,합계,1166901,957321,583847,14882,287222,20862,43719,152,111,655,103,2265,136,264,614,954832,2489,209580
7179,대전광역시,합계,1220602,945897,404545,191376,218769,59820,63669,1069,168,620,234,611,256,406,758,942301,3596,274705
7719,울산광역시,합계,941093,744960,282794,203602,128520,60289,62187,829,128,575,240,641,220,411,926,741362,3598,196133
8133,세종특별자치시,합계,189421,152801,77767,23211,32010,9192,9353,153,34,88,42,126,50,76,135,152237,564,36620
8234,경기도,합계,10262309,7916009,3319812,1637345,1807308,540023,546373,10778,1302,5058,1858,6139,2213,2883,6553,7887645,28364,2346300


In [34]:
df_19th.to_csv("temp1_president_19.csv", index=False, encoding="utf-8-sig")

# 18th_2012

In [35]:
# =============================================================================
# 18대 대통령선거 데이터 전처리 함수
# =============================================================================

import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : 깃허브 파일 URL

    Returns:
    --------
    raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xls"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xls"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_18th_presidential_election(file_path):
    """
    18대 대통령선거 데이터 전처리 함수

    엑셀 파일 구조:
    - 0~2행: 메타 정보 (무시)
    - 3행: 헤더 첫 번째 줄 (시도명, 구시군명 등)
    - 4행: 헤더 두 번째 줄 (후보자명, 득표수 등)
    - 5행부터: 실제 데이터

    특별한 필터링 로직:
    - '합계' 행: 모두 포함
    - '소계' 행: 구시군별로 첫 번째만 포함 (중복 제거)

    Parameters:
    -----------
    file_path : 파일 경로 (로컬 파일, 깃허브 blob URL, 또는 raw URL)
        - 로컬: "./data/18대_선거결과.xls"
        - 깃허브 blob: "https://github.com/user/repo/blob/main/파일.xls" (자동 변환됨)
        - 깃허브 raw: "https://github.com/user/repo/raw/main/파일.xls"

    Returns:
    --------
    전처리된 선거 데이터 (pandas DataFrame, 합계 및 소계 행만 포함)

    Example:
    --------
    # 깃허브에서 불러오기 (blob URL 그대로 사용 - 자동 변환됨)
    blob_url = "https://github.com/user/repo/blob/main/18대/선거결과.xls"
    df = process_18th_presidential_election(blob_url)

    # 로컬 파일 불러오기
    df = process_18th_presidential_election("./data/18대_선거결과.xls")
    """

    # ================================
    # 1. 파일 로드 (자동 감지)
    # ================================
    if file_path.startswith(('http://', 'https://')):
        # URL인 경우 - blob URL이면 raw URL로 변환
        raw_url = convert_github_url_to_raw(file_path)
        print(f"웹에서 파일 로드: {raw_url}")
        if raw_url != file_path:
            print(f"  (blob URL을 raw URL로 자동 변환)")
        df_all = pd.read_excel(raw_url, header=None)
    else:
        # 로컬 파일인 경우
        print(f"로컬에서 파일 로드: {file_path}")
        df_all = pd.read_excel(file_path, header=None)

    print(f"원본 데이터 크기: {df_all.shape}")

    # ================================
    # 2. 헤더 구성 (3행 + 4행 병합)
    # ================================
    # 파이썬 인덱스는 0부터 시작하므로:
    # 3행 = iloc[3] (실제 4번째 행)
    # 4행 = iloc[4] (실제 5번째 행)
    header_row_1 = df_all.iloc[3].fillna("")  # 시도명, 구시군명 등
    header_row_2 = df_all.iloc[4].fillna("")  # 후보자명, 득표수 등
    combined_headers = header_row_1.astype(str) + "_" + header_row_2.astype(str)

    # ================================
    # 3. 실제 데이터 추출 (5행부터)
    # ================================
    # 5행 = iloc[5] (실제 6번째 행)부터 데이터 시작
    df_data = df_all.iloc[5:].copy()
    df_data.columns = combined_headers

    print(f"데이터 추출 후 크기: {df_data.shape}")
    print(f"컬럼 수: {len(df_data.columns)}")

    # ================================
    # 4. 주요 컬럼 추출
    # ================================
    gugun_column = [col for col in df_data.columns if "구시군명" in col][0]
    dong_column = [col for col in df_data.columns if "읍면동명" in col][0]

    print(f"주요 컬럼:")
    print(f"  - 구시군: {gugun_column}")
    print(f"  - 읍면동: {dong_column}")

    # ================================
    # 5. 고급 필터링 (합계 + 소계 첫 번째)
    # ================================
    # 18대는 특별한 필터링 로직이 필요함

    # 5-1. 합계 행 찾기
    is_total = df_data[gugun_column].astype(str).str.contains("합계", na=False)

    # 5-2. 소계 행 찾기
    is_subtotal = df_data[dong_column].astype(str).str.contains("소계", na=False)

    # 5-3. 소계 중에서 구시군별로 첫 번째만 선택
    # (동일한 구시군에 여러 소계가 있을 경우 중복 제거)
    subtotal_first_idx = (
        df_data[is_subtotal]
        .groupby(df_data[gugun_column])  # 구시군별로 그룹화
        .head(1)                        # 각 그룹의 첫 번째만 선택
        .index                          # 인덱스 추출
    )

    # 5-4. 합계 인덱스와 소계 첫 번째 인덱스 결합
    total_idx = is_total[is_total].index      # 합계 행들의 인덱스
    final_idx = total_idx.union(subtotal_first_idx)  # 두 인덱스 합집합

    # 5-5. 원래 순서 유지하며 필터링된 행들만 추출
    df_filtered = df_data.loc[final_idx].copy()

    print(f"필터링 후 크기: {df_filtered.shape}")
    print(f"  - 합계 행: {len(total_idx)}개")
    print(f"  - 소계 행 (첫 번째만): {len(subtotal_first_idx)}개")

    # ================================
    # 6. 수치형 컬럼 정리
    # ================================
    # 4번째 컬럼(인덱스 4)부터는 보통 득표수 등 수치 데이터
    numeric_columns = df_filtered.columns[4:]

    print(f"수치형 컬럼 개수: {len(numeric_columns)}")

    for col in numeric_columns:
        # 쉼표 제거 → 공백 제거 → 빈값을 0으로 → 정수형 변환
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)           # 문자열로 변환
            .fillna("0")           # NaN 값을 "0" 문자열로 변경
            .str.replace(",", "")  # 천 단위 구분자 제거 (예: "1,234" → "1234")
            .str.strip()           # 앞뒤 공백 제거
            .replace("nan", "0")   # 'nan' 문자열을 "0"으로 변경
            .replace("", "0")      # 빈 문자열을 "0"으로 변경
            .astype(int)           # 최종적으로 정수형으로 변환
        )

    # ================================
    # 7. 결과 반환
    # ================================
    print(f"최종 데이터 크기: {df_filtered.shape}")
    print("전처리 완료!")

    return df_filtered


# =============================================================================
# 사용 예시
# =============================================================================

# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
# blob_url = "https://github.com/~~/korean-elections/blob/main/original/Presidential_Elections/18th_2012/제18대 대통령선거 개표자료.xls"
# df_18th = process_18th_presidential_election(blob_url)

# 로컬에서 불러오는 경우
# df_18th = process_18th_presidential_election("./data/18대_대선결과.xls")

# 업로드된 파일 사용하는 경우
# df_18th = process_18th_presidential_election("제18대 대통령선거 개표자료.xls")

# 결과 확인
# print(df_18th.head())
# print(df_18th.columns.tolist())

In [36]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url18 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Presidential_Elections/18th_2012/%EC%A0%9C18%EB%8C%80%20%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0%20%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C.xls"
df_18th = process_18th_presidential_election(blob_url18)

웹에서 파일 로드: https://github.com/sw1kwon/korean-elections/raw/main/original/Presidential_Elections/18th_2012/%EC%A0%9C18%EB%8C%80%20%EB%8C%80%ED%86%B5%EB%A0%B9%EC%84%A0%EA%B1%B0%20%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C.xls
  (blob URL을 raw URL로 자동 변환)
원본 데이터 크기: (18127, 16)
데이터 추출 후 크기: (18122, 16)
컬럼 수: 16
주요 컬럼:
  - 구시군: 구시군명_
  - 읍면동: 읍면동명_
필터링 후 크기: (246, 16)
  - 합계 행: 18개
  - 소계 행 (첫 번째만): 228개
수치형 컬럼 개수: 12
최종 데이터 크기: (246, 16)
전처리 완료!


In [37]:
df_18th

,_,시도명_,구시군명_,읍면동명_,투표구명_,선거인수_,투표수_,후보자별 득표수_새누리당\n박근혜,_민주통합당\n문재인,_무소속\n박종선,_무소속\n김소연,_무소속\n강지원,_무소속\n김순자,_계,무효\n투표수_,기권수_
5,NaN,전국,합계,NaN,0,40507842,30721459,15773128,14692632,12854,16687,53303,46017,30594621,126838,9786383
6,NaN,서울특별시,합계,NaN,0,8393847,6307869,3024572,3227639,3559,3793,11829,5307,6276699,31170,2085978
7,NaN,서울특별시,종로구,소계,0,141447,103189,49422,52747,73,86,211,97,102636,553,38258
71,NaN,서울특별시,중구,소계,0,115277,83095,40289,41919,67,56,151,71,82553,542,32182
136,NaN,서울특별시,용산구,소계,0,206665,147849,76997,69572,87,96,261,121,147134,715,58816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17763,NaN,경상남도,거창군,소계,0,51773,40103,28726,10466,61,22,75,244,39594,509,11670
17801,NaN,경상남도,합천군,소계,0,43981,33586,25313,7114,70,20,64,382,32963,623,10395
17848,NaN,제주특별자치도,합계,NaN,0,451731,330967,166184,161235,148,240,667,861,329335,1632,120764
17849,NaN,제주특별자치도,제주시,소계,0,328450,241552,119563,119622,94,164,473,570,240486,1066,86898


In [38]:
df_18th.columns.tolist()

['_',
 '시도명_',
 '구시군명_',
 '읍면동명_',
 '투표구명_',
 '선거인수_',
 '투표수_',
 '후보자별 득표수_새누리당\n박근혜',
 '_민주통합당\n문재인',
 '_무소속\n박종선',
 '_무소속\n김소연',
 '_무소속\n강지원',
 '_무소속\n김순자',
 '_계',
 '무효\n투표수_',
 '기권수_']

In [39]:
rename_dict18 = {
    '시도명_': '시도',
    '구시군명_': '구시군',
    '선거인수_': '선거인수',
    '투표수_': '투표수',
    '후보자별 득표수_새누리당\n박근혜': '득표수_1_새누리당_박근혜',
    '_민주통합당\n문재인': '득표수_2_민주통합당_문재인',
    '_무소속\n박종선': '득표수_4_무소속_박종선',
    '_무소속\n김소연': '득표수_5_무소속_김소연',
    '_무소속\n강지원': '득표수_6_무소속_강지원',
    '_무소속\n김순자': '득표수_7_무소속_김순자',
    '_계': '득표수_계',
    '무효\n투표수_': '무효투표수',
    '기권수_': '기권수'
}

In [40]:
df_18th.rename(columns=rename_dict18).drop(columns=['_','읍면동명_','투표구명_'])

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박근혜,득표수_2_민주통합당_문재인,득표수_4_무소속_박종선,득표수_5_무소속_김소연,득표수_6_무소속_강지원,득표수_7_무소속_김순자,득표수_계,무효투표수,기권수
5,전국,합계,40507842,30721459,15773128,14692632,12854,16687,53303,46017,30594621,126838,9786383
6,서울특별시,합계,8393847,6307869,3024572,3227639,3559,3793,11829,5307,6276699,31170,2085978
7,서울특별시,종로구,141447,103189,49422,52747,73,86,211,97,102636,553,38258
71,서울특별시,중구,115277,83095,40289,41919,67,56,151,71,82553,542,32182
136,서울특별시,용산구,206665,147849,76997,69572,87,96,261,121,147134,715,58816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17763,경상남도,거창군,51773,40103,28726,10466,61,22,75,244,39594,509,11670
17801,경상남도,합천군,43981,33586,25313,7114,70,20,64,382,32963,623,10395
17848,제주특별자치도,합계,451731,330967,166184,161235,148,240,667,861,329335,1632,120764
17849,제주특별자치도,제주시,328450,241552,119563,119622,94,164,473,570,240486,1066,86898


In [41]:
df_18th = df_18th.rename(columns=rename_dict18).drop(columns=['_','읍면동명_','투표구명_'])

In [42]:
df_18th[df_18th['구시군'] == '합계']

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박근혜,득표수_2_민주통합당_문재인,득표수_4_무소속_박종선,득표수_5_무소속_김소연,득표수_6_무소속_강지원,득표수_7_무소속_김순자,득표수_계,무효투표수,기권수
5,전국,합계,40507842,30721459,15773128,14692632,12854,16687,53303,46017,30594621,126838,9786383
6,서울특별시,합계,8393847,6307869,3024572,3227639,3559,3793,11829,5307,6276699,31170,2085978
2780,부산광역시,합계,2911700,2219699,1324159,882511,555,913,2878,2389,2213405,6294,692001
3938,대구광역시,합계,1990746,1585806,1267789,309034,366,624,2043,1984,1581840,3966,404940
4706,인천광역시,합계,2241366,1657821,852600,794213,508,1005,2730,1910,1652966,4855,583545
5551,광주광역시,합계,1117781,898416,69574,823737,268,333,1113,561,895586,2830,219365
6020,대전광역시,합계,1182321,904367,450576,448310,271,461,1291,969,901878,2489,277954
6466,울산광역시,합계,886061,694938,413977,275451,210,434,898,1463,692433,2505,191123
6812,세종특별자치시,합계,87707,64990,33587,30787,31,38,99,155,64697,293,22717
6866,경기도,합계,9364077,7018577,3528915,3442084,1997,3674,12577,7476,6996723,21854,2345500


In [43]:
df_18th.to_csv("temp1_president_18.csv", index=False, encoding="utf-8-sig")